# Carga de los id_costa en la colección de playas 

Este notebook realizar la carga de los id_costa en la colección de playas para poder hacer maching con la colección de costas y así obtener el tiempo en cada playa. Tambiés se obtiene la longitud y latitud de la costa. Estas coordenadas son de costas no de playas, así que alguna playa tiene coordenadas aproximadas, puesto que en una costa hay varias playas.

In [2]:
import pymongo
from pymongo import MongoClient
import pprint
import pandas as pd

Primero realizo alguna query sobre mongo para ver como funciona

In [5]:
pp = pprint.PrettyPrinter(indent=1)
client = MongoClient()
client = MongoClient('localhost', 27017)
db_aemet = client.AEMET
db_croja = client.CRUZROJA
col_playas = db_croja['playas']
col_localidades = db_aemet['localidades']
col_costas = db_aemet['costas']

In [6]:
#Número de localidades
resultado_localidades  = col_localidades.find({})
print resultado_localidades.count()

8117


In [7]:
#número de playas de la aemet
resultado_costas = col_costas.find({})
print resultado_costas.count()

591


In [8]:
#Número de playas de la cruz roja
resultado_playas = col_playas.find({})
print resultado_playas.count()

111


In [9]:
resultado  = col_localidades.find({"_id":u'312103'}).limit(1)
print resultado.count()
for doc in resultado:
    print  pp.pprint(doc)

0


In [10]:
i = 0
resultado  = col_localidades.find({"latitud":{ '$ne': ''}}).sort(u'nombre_provincia',1).limit(5)
for doc in resultado:
    
    poblacion_temperatura = doc["2015-07-01"]["temperatura"]
    print "Temperatura en " , doc["nombe_localidad"]  , ',',  doc["nombre_provincia"]
    print 'MAXIMA: ' ,  poblacion_temperatura['maxima']
    print 'MINIMA: ' ,  poblacion_temperatura['minima']
    print "LONGITUD: ", doc["longitud"]
    print "_ID:" , doc["_id"]
    print "uv_max" , doc.get("uv_max",'')
    i+=1

print "Numero de documentos sin coordenadas: " , i
    
        

KeyError: '2015-07-01'

In [149]:
resultado  = (col_costas
              .find({'nombre_provincia':'Alacant/Alicante'},
                    {'_id':1,'nombre_localidad':1,'nombre_playa':1,'nombre_provincia':1})
              )

for doc in resultado:
        print pp.pprint(doc)


{u'_id': u'0301101',
 u'nombre_localidad': u"Alf\xe0s del Pi, l'",
 u'nombre_playa': u'raco-de-l-albir',
 u'nombre_provincia': u'Alacant/Alicante'}
None
{u'_id': u'0301408',
 u'nombre_localidad': u'Alicante/Alacant',
 u'nombre_playa': u'el-postiguet',
 u'nombre_provincia': u'Alacant/Alicante'}
None
{u'_id': u'0301410',
 u'nombre_localidad': u'Alicante/Alacant',
 u'nombre_playa': u'saladar',
 u'nombre_provincia': u'Alacant/Alicante'}
None
{u'_id': u'0301401',
 u'nombre_localidad': u'Alicante/Alacant',
 u'nombre_playa': u'sant-joan-san-juan',
 u'nombre_provincia': u'Alacant/Alicante'}
None
{u'_id': u'0301809',
 u'nombre_localidad': u'Altea',
 u'nombre_playa': u'cap-blanch',
 u'nombre_provincia': u'Alacant/Alicante'}
None
{u'_id': u'0301808',
 u'nombre_localidad': u'Altea',
 u'nombre_playa': u'la-roda',
 u'nombre_provincia': u'Alacant/Alicante'}
None
{u'_id': u'0303102',
 u'nombre_localidad': u'Benidorm',
 u'nombre_playa': u'llevant-playa-de-levante',
 u'nombre_provincia': u'Alacant/Alica

In [50]:
resultado  = col_playas.find({}).limit(1)
for doc in resultado:
    print pp.pprint(doc)


{u'_id': 24,
 u'acceso_discapacitados': u'S\xed',
 u'atencion_cre': u'S\xed',
 u'atencion_discapacitados': u'S\xed',
 u'atencion_familiar': u'No',
 u'ayuda_banio': u'No',
 u'bandera': u'No hay informaci\xf3n',
 u'cantidad_puestos': u'2',
 u'cantidad_sillas_adaptadas': u'2',
 u'cantidad_torres_intervencion': u'0',
 u'cantidad_torres_vigilacia': u'2',
 u'cobertura_desde': u'06-06-2015',
 u'cobertura_hasta': u'13-09-2015',
 u'consejo': u'\r\n\t\t\t\t\t\t\t\tConsejo: \r\n\t\t\t\t\t\t\t',
 u'domingo': u'S\xed',
 u'duchas': u'S\xed',
 u'frecuencia_atencion': u'DIARIA',
 u'horario': u'11:00 - 18:00',
 u'horario_atencion': u'\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t11:00 / 15:00\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t',
 u'id': 24,
 u'jueves': u'S\xed',
 u'lunes': u'S\xed',
 u'martes': u'S\xed',
 u'medusas': u'No',
 u'miercoles': u'S\xed',
 u'municipio': u'Valencia',
 u'nombre_playa': u

## Empiezo a realizar la obtención de los ids

Desafortunadamente los nombres en las dos bases de datos no coinciden, así que hay que hacer el maching a mano. Esto vale con haerlo una vez. Lo primero es sacar los csvs de las dos bbdd

In [56]:
#Voy a crear un csv con los nombres e ids de las playas para hacer matching con las de la AEMET

resultado_playas = col_playas.find({},{'_id':1,'nombre_playa':1, 'provincia':1})
df_playas = pd.DataFrame(list(resultado_playas))
df_playas.to_csv('playas_cruzroja.csv',index = False, encoding='utf-8')


In [87]:
resultado_costas = col_costas.find({},{'_id':1,'nombre_playa':1, 'nombre_provincia':1, 'nombre_localidad':1})
df_costas = pd.DataFrame(list(resultado_costas))
df_costas.to_csv('playas_aemet.csv',index = False, encoding='utf-8')

Esto es una prueba para ver cuantos hacen maching, como vemos son muy pocos y es por ello que hay que hacerlo a mano. Solo se hace en principio para Cantabria y Comunidad Valenciana (Valencia, Alicante y Castellón)

In [83]:
i = 0

for id_cr in df_playas.index:
    playa_cr = df_playas.iloc[id_cr].nombre_playa.strip().lower()
    playa_cr_id = df_playas.iloc[id_cr]._id
    provincia_cr = df_playas.iloc[id_cr].provincia.strip().lower() 
    for id_ae in df_costas.index:
        playa_ae = df_costas.iloc[id_ae].nombre_playa.strip().lower()
        playa_ae_id = df_costas.iloc[id_ae]._id
        provincia_ae = df_costas.iloc[id_ae].nombre_provincia.strip().lower()
        if playa_ae.find(playa_cr) >0:
            print "Bien: " , playa_cr , " en ", provincia_cr , provincia_ae
            i+=1
            break
            
    
print "Hay ", i , " bien"

Bien:  zurriola  en  gipuzkoa gipuzkoa
Bien:  tarajal  en  ceuta las palmas
Bien:  toranda  en  asturias asturias
Bien:  santiago  en  gipuzkoa gipuzkoa
Bien:  roqueta  en  alicante alacant/alicante
Bien:  raco  en  girona valència/valencia
Bien:  puerto  en  gipuzkoa alacant/alicante
Bien:  puerto  en  murcia alacant/alicante
Bien:  poniente  en  murcia alacant/alicante
Bien:  palmera  en  asturias asturias
Bien:  meron  en  cantabria cantabria
Bien:  nares  en  murcia murcia
Bien:  llevant  en  barcelona illes balears
Bien:  levante  en  alicante alacant/alicante
Bien:  levante  en  murcia alacant/alicante
Bien:  granadella  en  alicante alacant/alicante
Bien:  gorrondatxe  en  bizkaia/vizcaya bizkaia
Bien:  gorliz  en  bizkaia/vizcaya bizkaia
Bien:  ea  en  bizkaia/vizcaya a coruña
Bien:  colonia  en  murcia murcia
Bien:  chorrillo  en  ceuta cádiz
Bien:  bogatell  en  barcelona barcelona
Bien:  barro  en  asturias asturias
Bien:  antilla  en  gipuzkoa gipuzkoa
Bien:  almadrava  en 

Una vez tenemos el maching hecho en el csv playas_cruzroja.csv lo cargamos para llevarlo a la bbdd de mongo

# A PARTIR DE AQÍ MACHING

In [11]:
# Vamos a cargar los ids de las costas en la en la bbdd de mongo de playas
# Lo primero es cargar el csv que tiene los ids, luego filtrar por aquellos que tienen el id relleno y por
# último actualizar los registros en la bbdd de playas

df_playas = pd.DataFrame.from_csv('playas_cruzroja.csv',)
df_playas = df_playas.dropna()


In [12]:
#y ahora inserto en mongo en la db = CRUZROJA , coleccion = playas , el id_costa para el _id de la playa en el dataframe
for id_cr in df_playas.index:
    id_playa = id_cr
    id_costa = int(df_playas.ix[id_cr].id_costa)
    costa = {}
    costa["id_costa"] = id_costa
    post_id = col_playas.update({"_id" : id_playa},{ "$set" : costa}, upsert=True)
    print post_id
    

{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'updatedExisting': True, u'nModified': 1, u'ok': 1, u'n': 1}
{'update

In [13]:
#Hago una query de comprobación (saco unicamente un registro):
resultado  = col_playas.find({"id_costa": { "$exists" : True }},
                             {"nombre_playa":1,
                              "id_costa":1,
                              "medusas":1,
                              "municipio":1,
                              "provincia":1})
for doc in resultado:
    print "El documento de playas:"
    print pp.pprint(doc)
    id_costa = str(doc["id_costa"])
    
    if len(id_costa)==6:
        id_costa = "0"+id_costa
    

        
    print "El documento de costas para esa playa:"
    resulcosta = col_costas.find({"_id": unicode(id_costa)}, 
                                 {"nombre_playa":1, 
                                  "nombre_localidad":1, 
                                  "nombre_provincia":1 , 
                                  "longitud":1,
                                  "latitud":1})
    for doc_costa in resulcosta:
        print pp.pprint(doc_costa)
        
    
    print "---------------------------------------------------------------------------------------------"
    print
    
    

El documento de playas:
{u'_id': 1128, u'id_costa': 3908004}
None
El documento de costas para esa playa:
{u'_id': u'3908004',
 u'latitud': u'-4.36666666667',
 u'longitud': u'43.3938888889',
 u'nombre_localidad': u'San Vicente de la Barquera',
 u'nombre_playa': u'el-sable-de-meron',
 u'nombre_provincia': u'Cantabria'}
None
---------------------------------------------------------------------------------------------

El documento de playas:
{u'_id': 1127, u'id_costa': 3908004}
None
El documento de costas para esa playa:
{u'_id': u'3908004',
 u'latitud': u'-4.36666666667',
 u'longitud': u'43.3938888889',
 u'nombre_localidad': u'San Vicente de la Barquera',
 u'nombre_playa': u'el-sable-de-meron',
 u'nombre_provincia': u'Cantabria'}
None
---------------------------------------------------------------------------------------------

El documento de playas:
{u'_id': 1120, u'id_costa': 3909101}
None
El documento de costas para esa playa:
{u'_id': u'3909101',
 u'latitud': u'-4.33194444444',
 u'l

In [16]:
playas = col_playas.find({"_id": 1128})
for playa in playas:
    print playa

{u'latitud': u'-4.36666666667', u'_id': 1128, u'longitud': u'43.3938888889', u'id_costa': 3908004}
